In [1]:
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2
%matplotlib inline 


import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from scipy import stats
from scipy.interpolate import interp1d,interp2d
from scipy.optimize import least_squares
import pandas as pd
import sys
from tqdm import tqdm

# Import code
sys.path.insert(0, './../../../Scripts/scripts_unstructured/')
import functions as fun


# Simulation parameters

In [2]:
## Parameters that come from model fitting
best_param=np.array([ 9.97,  8.4 , -0.48, -1.7 , -0.1 , -3.88])
best_inputs=np.array([0.86, 0.89, 0.91, 0.94, 0.95, 1.02, 1.14, 0.62])


# parameters characterizing the relative strength of synaptic connections
g_E=best_param[0]; g_I=best_param[1];beta=10**best_param[2]; 

# average number of E connections per neuron (I connections are 1/4 of that)
K=500; 

# ratio between mean and standard deviation of the opsin expression
sigma_Lambda_over_Lambda=10**best_param[4]; 

# CV_K
CV_K=10**best_param[3]

# Synaptic efficacy in Volts
J=10**best_param[5]; 

#  Laser intensity that fits the data
L=[0., 10**best_inputs[-1]]; 

# FF inputs
r_X=10**best_inputs[0:-1]




In [3]:

# probability of connection between cells. 
p=0.1

# Presynaptic population has mean rate r_X, Gaussianly distributed with std=0.2* r_X
sigma_X_over_r_X=0.2

# mean opsin expression in Volt
Lambda=1*10**-3; 
sigma_Lambda=sigma_Lambda_over_Lambda*Lambda

# Compute W_{AB} starting from the parameters defined above

# Gain of Excitatory and inhibitory cells and I cells
G_E,G_I=1.0,2.0 

w_EE=1;w_IE=w_EE/beta;
w_EI=g_E*w_EE;w_II=g_I*w_IE;
w_EX,w_IX=(G_I*fun.gamma*g_E-G_E)*w_EE,(G_I*fun.gamma*g_I-G_E)*w_IE; 
w_X=np.asarray([w_EX,w_IX]);
w=np.zeros((2,2));
w[0,:]=w_EE,-w_EI
w[1,:]=w_IE,-w_II


# Obtain the LIF nonlinearity

In [4]:
# Input noise, determine how smooth the single neuron transfer function is 
sigma_t=0.01;

# Refractory period
tau_rp=0.002; 


#'''
mu_tab_max=10.0;
mu_tab=np.linspace(-mu_tab_max,mu_tab_max,200000)
mu_tab=np.concatenate(([-10000],mu_tab))
mu_tab=np.concatenate((mu_tab,[10000]))

phi_tab_E,phi_tab_I=mu_tab*0,mu_tab*0;
for idx in range(len(phi_tab_E)):
    phi_tab_E[idx]=fun.comp_phi_tab(mu_tab[idx],fun.tau_E,tau_rp,sigma_t)
    phi_tab_I[idx]=fun.comp_phi_tab(mu_tab[idx],fun.tau_I,tau_rp,sigma_t)

phi_int_E=interp1d(mu_tab, phi_tab_E, kind='linear')  
phi_int_I=interp1d(mu_tab, phi_tab_I, kind='linear')
#'''

# Generate quenched disorder

In [5]:

M,mu_X_over_r_X_tau,Lambda_i,N_E,N_I=fun.Generate_quenched_disorder(CV_K,J,K,w,w_X,p,Lambda,sigma_Lambda)


# Computing and saving network response

In [6]:

# Time in units of E time constant
Tmax_over_tau_E=200;

T=np.arange(0,Tmax_over_tau_E*fun.tau_E,fun.tau_I/3);

mask_time=T>(10*fun.tau_E)
RATES=-1*np.ones((len(r_X),len(L),N_E+N_I))
DYNA=-1*np.ones((len(r_X),len(L),N_E+N_I,len(T)))
MUS=-1*np.ones((len(r_X),len(L),N_E+N_I))
Lambda_i_L=1./tau_rp*np.ones((len(r_X),len(L),N_E+N_I))
MFT_SOL_R=np.ones((len(r_X),len(L),10))
MFT_SOL_M=np.ones((len(r_X),len(L),10))
Phi=[phi_int_E,phi_int_I];
for idx_r_X in tqdm(range(len(r_X))):
    for idx_L in range(len(L)):
        r_X_local,L_local=r_X[idx_r_X],L[idx_L];

        DYNA[idx_r_X,idx_L,:,:], MUS[idx_r_X,idx_L,:],Lambda_i_L[idx_r_X,idx_L,:]=fun.High_dimensional_dynamics(T,L_local,r_X_local,M,mu_X_over_r_X_tau,Lambda_i,N_E,N_I,phi_int_E,phi_int_I);
        RATES[idx_r_X,idx_L,:]=np.mean(DYNA[idx_r_X,idx_L,:,mask_time],axis=0)

    Base_Sim=RATES[idx_r_X,0,:]
    Delta_Sim=RATES[idx_r_X,idx_L,:]-RATES[idx_r_X,0,:]

    moments_of_r_sim=np.zeros(5)
    moments_of_r_sim[0]=np.mean(Base_Sim)
    moments_of_r_sim[1]=np.mean(Delta_Sim)
    moments_of_r_sim[2]=np.std(Base_Sim)
    moments_of_r_sim[3]=np.std(Delta_Sim)
    moments_of_r_sim[4]=np.cov(Base_Sim,Delta_Sim)[0,1]
#     print(moments_of_r_sim[:])


100%|██████████| 7/7 [15:32<00:00, 133.20s/it]


In [8]:
np.save('RATES_unstructured_mouse.txt',RATES)
